<a href="https://colab.research.google.com/github/Paulflow-dev/Falcon-9/blob/master/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas dash


In [3]:
!pip install packaging

In [ ]:
!pip install httpx==0.20 dash plotly

In [4]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc
from dash import html

import plotly.graph_objects as go
from dash.dependencies import Input, Output

In [5]:
data=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv",index_col=0)

In [6]:
data.head(3)

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0


In [ ]:
#Initialize the app
app = dash.Dash(__name__)

# Define layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
                                html.P(),
                                html.Div(["Launch-Site: ",dcc.Dropdown(id='site-dropdown', options=[{'label': 'All Sites', 'value': 'ALL'},{'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},{'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},{'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},{'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},],value='ALL',placeholder="place holder here",searchable=True)], style={'height':'50px', 'font-size': 15}),
                                html.Div(dcc.Graph(id='success-pie-chart')),

                                html.Div(["Range Slicer: ",dcc.RangeSlider(id='load-range',min=0, max=10000, step=1000,marks={0: '0',1000: '100',2000:'2000',5000:'5000'},value=[0,1000])]),
                                html.Div(dcc.Graph(id='success-scatter-chart')),

                    ])

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_data= data[data['class'] == 1]
    if entered_site == 'ALL':
        fig = px.pie(filtered_data, values='class',
        names='Launch Site',
        title='Successful Launches')
        return fig
    else:
        if entered_site != 'ALL' :
          fig = px.pie(filtered_data[filtered_data['Launch Site']==entered_site], values='class',
          names='Launch Site',
          title='Successful launches for  '+ str(entered_site))
          return fig

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="load-range", component_property="value")])

def get_scatter_chart(entered_site,entered_load):
    filtered_data=data[(data['Payload Mass (kg)'] >= entered_load[0]) & (data['Payload Mass (kg)'] <= entered_load[1])]

    if entered_site == 'ALL':
      fig2=px.scatter(filtered_data, x='Payload Mass (kg)',y='class',color='Booster Version Category')
      return fig2

    else:
        if entered_site != 'ALL' :
          filtered_data_2=filtered_data[filtered_data['Launch Site']==entered_site]
          fig2=px.scatter(filtered_data_2, x='Payload Mass (kg)',y='class',color='Booster Version Category')
          return fig2

#Run the app
if __name__ == '__main__':
    app.run_server()